Now we will continue on the [Conversation AI](https://conversationai.github.io/) dataset seen in [week 4 homework and lab](https://github.com/MIDS-scaling-up/v2/tree/master/week04). 
We shall use a version of pytorch BERT for classifying comments found at [https://github.com/huggingface/pytorch-pretrained-BERT](https://github.com/huggingface/pytorch-pretrained-BERT).  

The original implementation of BERT is optimised for TPU. Google released some amazing performance improvements on TPU over GPU, for example, see [here](https://medium.com/@ranko.mosic/googles-bert-nlp-5b2bb1236d78) - *BERT relies on massive compute for pre-training ( 4 days on 4 to 16 Cloud TPUs; pre-training on 8 GPUs would take 40–70 days).*. In response, Nvidia released [apex](https://devblogs.nvidia.com/apex-pytorch-easy-mixed-precision-training/), which gave mixed precision training. Weights are stored in float32 format, but calculations, like forward and backward propagation happen in float16 - this allows these calculations to be made with a [4X speed up](https://github.com/huggingface/pytorch-pretrained-BERT/issues/149).  

We shall apply BERT to the problem for classifiying toxicity, using apex from Nvidia. We shall compare the impact of hardware by running the model on a V100 and P100 and comparing the speed and accuracy in both cases.   

This script relies heavily on an existing [Kaggle kernel](https://www.kaggle.com/yuval6967/toxic-bert-plain-vanila) from [yuval r](https://www.kaggle.com/yuval6967). 
  
*Disclaimer: the dataset used contains text that may be considered profane, vulgar, or offensive.*

In [1]:
# Download the training and the test corpus
#!wget -nv --show-progress -O data/test.csv.zip https://www.dropbox.com/s/xp6bo8yo1vbv5yg/test.csv.zip?dl=1
#!wget -nv --show-progress -O data/train.csv.zip https://www.dropbox.com/s/xei6z41mfrcnxcd/train.csv.zip?dl=1
# Download the pretrained weights for bert base. 
#!wget -nv --show-progress -O data/uncased_L-12_H-768_A-12.zip \
#        https://storage.googleapis.com/bert_models/2018_10_18/uncased_L-12_H-768_A-12.zip
#!wget -nv --show-progress  -O data/cased_L-12_H-768_A-12.zip \
#        https://storage.googleapis.com/bert_models/2018_10_18/cased_L-12_H-768_A-12.zip
# unzip weights & conifg and remove the original zip
#!unzip -d data/ data/cased_L-12_H-768_A-12.zip && rm data/cased_L-12_H-768_A-12.zip
#!unzip -d data/ data/uncased_L-12_H-768_A-12.zip && rm data/uncased_L-12_H-768_A-12.zip

In [2]:
import sys, os
import numpy as np 
import pandas as pd 
import torch
import torch.nn as nn
import torch.utils.data
import torch.nn.functional as F
from sklearn.metrics import roc_auc_score

%load_ext autoreload
%autoreload 2
%matplotlib inline
from tqdm import tqdm, tqdm_notebook
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
import warnings
warnings.filterwarnings(action='once')
import pickle
import shutil

In [3]:
# Let's activate CUDA for GPU based operations
device=torch.device('cuda')

Change the PATH variable to whereever your `week06/hw` directory is located.  
**For the final run we would like you to have a train_size of at least 1 Million rows, and a valid size of at least 500K rows. When you first run the script, feel free to work with a reduced train and valid size for speed.** 

In [4]:
# In bert we need all inputs to have the same length, we will use the first 220 characters. 
MAX_SEQUENCE_LENGTH = 220
SEED = 1234
# We shall run a single epoch (ie. one pass over the data)
EPOCHS = 2
PATH = '.'#'/root/v2/week06/hw' # /root/v2/week06/hw"
DATA_DIR = os.path.join(PATH, "data")
WORK_DIR = os.path.join(PATH, "workingdir")

# Validation and training sizes are here. 
#train_size= 10000 # 1000000 
#valid_size= 5000  # 500000
train_size= 1000000 
valid_size= 500000

This should be the files you downloaded earlier when you ran `download.sh`

In [5]:
os.listdir(DATA_DIR)

['train.csv.zip',
 'test.csv.zip',
 'uncased_L-12_H-768_A-12',
 'cased_L-12_H-768_A-12',
 'tmp']

We shall install pytorch BERT implementation.   
If you would like to experiment with or view any code (purely optional, and not graded :) ), you can copy the files from the repo https://github.com/huggingface/pytorch-pretrained-BERT  

In [6]:
%%capture
from transformers import BertModel, BertConfig, BertTokenizer, BertForSequenceClassification
from transformers import AdamW as BertAdam

We shall now load the model. When you run this, comment out the `capture` command to understand the archecture.

In [7]:
#%%capture
bert_config = BertConfig()

Bert needs a special formatting of sentences, so we have a sentence start and end token, as well as separators.   
Thanks to this [script](https://www.kaggle.com/httpwwwfszyc/bert-in-keras-taming) for a fast convertor of the sentences.

In [8]:
def convert_lines(example, max_seq_length,tokenizer):
    max_seq_length -=2
    all_tokens = []
    longer = 0
    for text in tqdm_notebook(example):
        tokens_a = tokenizer.tokenize(text)
        if len(tokens_a)>max_seq_length:
            tokens_a = tokens_a[:max_seq_length]
            longer += 1
        one_token = tokenizer.convert_tokens_to_ids(["[CLS]"]+tokens_a+["[SEP]"])+[0] * (max_seq_length - len(tokens_a))
        all_tokens.append(one_token)
    print(longer)
    return np.array(all_tokens)

Now we load the BERT tokenizer and convert the sentences.

In [9]:
%%time
# tokenizer = BertTokenizer.from_pretrained(BERT_MODEL_PATH, cache_dir=None,do_lower_case=True)
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
train_all = pd.read_csv(os.path.join(DATA_DIR, "train.csv.zip")).sample(train_size+valid_size,random_state=SEED)
print('loaded %d records' % len(train_all))

# Make sure all comment_text values are strings
train_all['comment_text'] = train_all['comment_text'].astype(str) 

sequences = convert_lines(train_all["comment_text"].fillna("DUMMY_VALUE"),MAX_SEQUENCE_LENGTH,tokenizer)
train_all=train_all.fillna(0)

loaded 1500000 records


/home/ubuntu/anaconda3/envs/pytorch_latest_p36/lib/python3.6/site-packages/ipykernel_launcher.py:5: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  """



33722
CPU times: user 28min 31s, sys: 7.82 s, total: 28min 39s
Wall time: 28min 31s


Let us look at how the tokenising works in BERT, see below how it recongizes misspellings - words the model never saw. 

In [10]:
train_all[["comment_text", 'target']].head()

,comment_text,target
458232,It's difficult for many old people to keep up ...,0.000000
272766,She recognized that her tiny-handed husband is...,0.166667
339129,"HPHY76,\nGood for you for thinking out loud, w...",0.000000
773565,And I bet that in the day you expected your Je...,0.500000
476233,Kennedy will add a much needed and scientifica...,0.000000


Lets tokenize some text (I intentionally mispelled some words to check berts subword information handling)

In [11]:
text = 'Hi, I am learning new things in w251 about deep learning the cloud and teh edge.'
tokens = tokenizer.tokenize(text)
' '.join(tokens)

'hi , i am learning new things in w ##25 ##1 about deep learning the cloud and te ##h edge .'

Added start and end token and convert to ids. This is how it is fed into BERT.

In [12]:
tokens = ["[CLS]"] + tokens + ["[SEP]"]
input_ids = tokenizer.convert_tokens_to_ids(tokens)
' '.join(map(str, input_ids))

'101 7632 1010 1045 2572 4083 2047 2477 1999 1059 17788 2487 2055 2784 4083 1996 6112 1998 8915 2232 3341 1012 102'

When BERT converts this sentence to a torch tensor below is shape of the stored tensors.  
We have 12 input tensors, while the sentence tokens has length 23; where are can you see the 23 tokens in the tensors ?... **Feel free to post in slack or discuss in class**

In [13]:
# put input on gpu and make prediction
bert = BertForSequenceClassification.from_pretrained('bert-base-uncased').cuda()
# bert = BertModel.from_pretrained(WORK_DIR).cuda()
bert_output = bert(torch.tensor([input_ids]).cuda())

print('Sentence tokens {}'.format(tokens))
print('Number of tokens {}'.format(len(tokens)))
print('Tensor shapes : {}'.format([b.cpu().detach().numpy().shape for b in bert_output[0]]))
print('Number of torch tensors : {}'.format(len(bert_output[0])))

Sentence tokens ['[CLS]', 'hi', ',', 'i', 'am', 'learning', 'new', 'things', 'in', 'w', '##25', '##1', 'about', 'deep', 'learning', 'the', 'cloud', 'and', 'te', '##h', 'edge', '.', '[SEP]']
Number of tokens 23
Tensor shapes : [(2,)]
Number of torch tensors : 1


As it is a binary problem, we change our target to [0,1], instead of float.   
We also split the dataset into a training and validation set, 

In [14]:
train_all['target']=(train_all['target']>=0.5).astype(float)
# Training data - sentences
X = sequences[:train_size] 
# Target - the toxicity. 
y = train_all[['target']].values[:train_size]
X_val = sequences[train_size:]                
y_val = train_all[['target']].values[train_size:]

In [15]:
test_df=train_all.tail(valid_size).copy()
train_df=train_all.head(train_size)

**From here on in we would like you to run BERT.**   
**Please do rely on the script available -  [Kaggle kernel](https://www.kaggle.com/yuval6967/toxic-bert-plain-vanila) from [yuval r](https://www.kaggle.com/yuval6967) - for at least the first few steps up to training and prediction.**  
    
**Note, instead of using the apex in yuval's code, we would like you to use the new native [pytorch apex version](https://pytorch.org/docs/stable/amp.html).** 
**You can see some examples of native [apex](https://pytorch.org/docs/stable/notes/amp_examples.html#typical-mixed-precision-training) here.**


**1)**   
**Load the training set to a training dataset. For this you need to load the X sequences and y objects to torch tensors**   
**You can use `torch.utils.data.TensorDataset` to input these into a train_dataset.**

In [16]:
# Training data creations
train_dataset = torch.utils.data.TensorDataset(torch.tensor(X,dtype=torch.long), torch.tensor(y,dtype=torch.float))


**2)**  
**Set your learning rate and batch size; and optionally random seeds if you want reproducable results**   
**Load your pretrained BERT using `BertForSequenceClassification`**   
**Initialise the gradients and place the model on cuda, set up your optimiser and decay parameters**
**note, we will not use apex like yuval. So no `amp.initialise` needed.**

In [17]:
lr=1e-5
batch_size = 32
accumulation_steps=2
print(sequences)

[[  101  2009  1005 ...     0     0     0]
 [  101  2016  3858 ...     0     0     0]
 [  101  6522 10536 ...  2123  1005   102]
 ...
 [  101  2531  1003 ...     0     0     0]
 [  101  2928  2017 ...     0     0     0]
 [  101  2008  1005 ...     0     0     0]]


In [18]:
%%time
model = BertForSequenceClassification.from_pretrained('bert-base-uncased',num_labels=1)
model.zero_grad()
model = model.to(device)
param_optimizer = list(model.named_parameters())
no_decay = ['bias', 'LayerNorm.bias', 'LayerNorm.weight']
optimizer_grouped_parameters = [
    {'params': [p for n, p in param_optimizer if not any(nd in n for nd in no_decay)], 'weight_decay': 0.01},
    {'params': [p for n, p in param_optimizer if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
    ]
train = train_dataset

num_train_optimization_steps = int(EPOCHS*len(train)/batch_size/accumulation_steps)

optimizer = BertAdam(optimizer_grouped_parameters, lr=lr)

model=model.train()

CPU times: user 5.01 s, sys: 571 ms, total: 5.58 s
Wall time: 4.64 s


**3)**  
**Start training your model by iterating through batches in a single epoch of the data**    
**I copied the new amp process shown [here](https://github.com/aws/deep-learning-containers/blob/b9ccd5928fe2ecd8aafc4c1660b39fe265ed879f/test/dlc_tests/container_tests/bin/pytorch_tests/testPyTorchAMP#L31-L40) and it worked out.**   
A few tips on the changes needed,   
1) You will need to use `torch.cuda.amp.autocast()` and `torch.cuda.amp.GradScaler()` like seen in the reference code. 
2) From yuval's code, the prediction and loss calculation must be executed with autocast (see new amp process code).      
3) Loss scaling, the omptimiser step and update can be taken from the new amp code, but please do the optimiser step and update with accumulation from yuval's code.     
4) For some reason, yuval's code, in the `y_pred` line, returns a tuple when run with autocast. Just index into the first element to get the prediction.    

In [19]:
%%time
from torch.cuda.amp import autocast, GradScaler

tq = tqdm_notebook(range(EPOCHS))

# Creates a GradScaler once at the beginning of training.
scaler = GradScaler()

for epoch in tq:
    train_loader = torch.utils.data.DataLoader(train, batch_size=batch_size, shuffle=True)
    avg_loss = 0.
    avg_accuracy = 0.
    lossf=None
    tk0 = tqdm_notebook(enumerate(train_loader),total=len(train_loader),leave=False)
    optimizer.zero_grad()   # Bug fix - thanks to @chinhuic
    for i,(x_batch, y_batch) in tk0:
        with autocast():
            y_pred = model(x_batch.to(device), attention_mask=(x_batch>0).to(device), labels=None)[0]
            loss =  F.binary_cross_entropy_with_logits(y_pred,y_batch.to(device))
        scaler.scale(loss).backward()
        if (i+1) % accumulation_steps == 0:             # Wait for several backward steps
            scaler.step(optimizer)                      # Now we can do an optimizer step
            scaler.update()
            optimizer.zero_grad()
        if lossf:
            lossf = 0.98*lossf+0.02*loss.item()
        else:
            lossf = loss.item()
        tk0.set_postfix(loss = lossf)
        avg_loss += loss.item() / len(train_loader)
        avg_accuracy += torch.mean(((torch.sigmoid(y_pred[:,0])>0.5) == (y_batch[:,0]>0.5).to(device)).to(torch.float) ).item()/len(train_loader)
    tq.set_postfix(avg_loss=avg_loss,avg_accuracy=avg_accuracy)

/home/ubuntu/anaconda3/envs/pytorch_latest_p36/lib/python3.6/site-packages/ipykernel_launcher.py:3: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  This is separate from the ipykernel package so we can avoid doing imports until



CPU times: user 2h 32min 36s, sys: 30min, total: 3h 2min 36s
Wall time: 3h 1min 42s


**4)**  
**Store your trained model to disk, you will need it if you choose section 8C.**

In [20]:
output_model_file = "bert_pytorch.bin"
torch.save(model.state_dict(), output_model_file)


**5)**   
**Now make a prediction for your validation set.**  

In [21]:
model.to(device)
for param in model.parameters():
    param.requires_grad=False
model.eval()
valid_preds = np.zeros((len(X_val)))
valid = torch.utils.data.TensorDataset(torch.tensor(X_val,dtype=torch.long))
valid_loader = torch.utils.data.DataLoader(valid, batch_size=32, shuffle=False)

tk0 = tqdm_notebook(valid_loader)
for i,(x_batch,)  in enumerate(tk0):
    pred = model(x_batch.to(device), attention_mask=(x_batch>0).to(device), labels=None)[0]
    valid_preds[i*32:(i+1)*32]=pred[:,0].detach().cpu().squeeze().numpy()

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, element

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, element

/home/ubuntu/anaconda3/envs/pytorch_latest_p36/lib/python3.6/site-packages/ipykernel_launcher.py:9: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  if __name__ == '__main__':


**6)**  
**In the yuval's kernel he get a metric based on the metric for the jigsaw competition - it is quite complicated. Instead, we would like you to measure the `AUC`, similar to how you did in homework 04. You can compare the results to HW04**  
*A tip, if your score is lower than homework 04 something is wrong....*

In [22]:
print('AUC score : {:.5f}'.format(roc_auc_score(y_val, valid_preds)))

AUC score : 0.96891


**7)**  
**Can you show/print the validation sentences predicted with the highest and lowest toxicity ?**

In [23]:
print("Most toxic")
print(80*'=')
lowest = valid_preds.argsort()[-3:][::-1]
for idx in lowest:
    print(train_all.iloc[train_size+idx]['comment_text'])
    print(80*'-')
    
print("\nLeast toxic")
print(80*'=')
highest = valid_preds.argsort()[:3][::-1]
for idx in highest:
    print(train_all.iloc[train_size+idx]['comment_text'])
    print(80*'-')

Most toxic
As big of an idiot that Trump is, I'd rather have a moron that can't get anywhere with Congress than World War 3.

"Trump seemed to argue for a hands-off policy in Syria, effectively conceding that the rebel-held portions of Aleppo – which are under heavy bombardment from Syrian government forces and their Russian allies – were already lost. "It has fallen, I mean, from any standpoint. What do you need, a signed document? I mean, from any standpoint," Trump said.

In the end, Trump's argument for U.S. policy in Syria was to leave Assad alone, or seek to partner with him and the Russians to fight the Islamic State.

"He's a bad guy. But you may very well end up with worse than Assad" if he was ever deposed. "

He may be an idiot, but even an idiot could see what a mess Iraq is and would stop wasting our treasury killing people in the middle east.
--------------------------------------------------------------------------------
Let's just blast him from the planet and be done w

**8)**  
**Pick only one of the below items and complete it. The last two will take a good amount of time (and partial success on them is fine), so proceed with caution on your choice of items :)** 
  
  
**A. Can you train on two epochs ?**

Yes, I've set the training to two epochs here.

**B. Can you change the learning rate and improve validation score ?**

Yes, I cut the learning rate in half (2e-5 => 1e-5) and that improved AUC by 0.014 (on the development sets).
   
**C. Make a prediction on the test data set with your downloaded model and submit to Kaggle to see where you score on public LB - check out [Abhishek's](https://www.kaggle.com/abhishek) script - https://www.kaggle.com/abhishek/pytorch-bert-inference . Note, you will need to fork Abhisheks kernel, swap out the weights to your downloaded weights and commit the kernel. When finalised and you get the output, there is a button to submit to the competition**  
  
**D. Get BERT running on the tx2 for a sample of the data.** 
  
**E. Finally, and very challenging -- the `BertAdam` optimiser proved to be suboptimal for this task. There is a better optimiser for this dataset in this script [here](https://www.kaggle.com/cristinasierra/pretext-lstm-tuning-v3). Check out the `custom_loss` function. Can you implement it ? It means getting under the hood of the `BertForSequenceClassification` at the source repo and implementing a modified version locally .  `https://github.com/huggingface/pytorch-pretrained-BERT/blob/master/pytorch_pretrained_bert/modeling.py`**

### **Remember to terminate your instance at the end.**